In [1]:
import pandas as pd
import numpy as np

In [ ]:
Задание 3

In [ ]:
Выгрузите организации, где число сотрудников в 2020 году 15 и больше. 
Если за 2020 год данных по сотрудникам нет, то возьмите за 2019 год. 
Отсортируйте по возрастанию числа сотрудников. Если число сотрудников 
одинаковое, то организации должны идти по убыванию ID.

In [181]:
# создаем подключение к базе данных:

import sqlite3

con = sqlite3.connect('indicators_db.sqlite3')

In [182]:
con.execute('select * from org_indicators_2020_2022')

In [183]:
# обращаемся к объекту подключения и прописываем запрос
# присваеваем резултат переменной 2020_2022

cursor = con.execute('select * from org_indicators_2020_2022')

In [184]:
# обращаемся к объекту подключения и прописываем запрос
# присваеваем резултат переменной 2019

cursor1 = con.execute('select * from org_indicators_2019')

In [185]:
# вытягиваем наши данные из объекта курсора <sqlite3.Cursor at 0x16ad20d32c0>

row = cursor.fetchall()

row1 = cursor1.fetchall()

df_all_indicators = pd.DataFrame(row, columns = ['ID', 'Year', 'Indicator', 'Value'])

df_all_indicators1 = pd.DataFrame(row1, columns = ['ID', 'Year', 'Indicator', 'Value'])

In [186]:
df_2020 = df_all_indicators[(df_all_indicators['Year'] == 2020) & (df_all_indicators['Indicator'] == 'Число сотрудников')]

In [187]:
df_2019 = df_all_indicators1[(df_all_indicators1['Year'] == 2019) & (df_all_indicators1['Indicator'] == 'Число сотрудников')]

In [188]:
df_2020 = df_2020.reindex(['ID', 'Year', 'Value'], axis = 1)

In [189]:
df_2019 = df_2019.reindex(['ID', 'Year', 'Value'], axis = 1)

In [190]:
df = df_2019.pivot(values = 'Value', columns = 'Year', index = 'ID').reset_index()

In [191]:
df1 = df_2020.pivot(values = 'Value', columns = 'Year', index = 'ID').reset_index()

In [155]:
df = pd.merge(df, df1, on = "ID", how = 'right')

In [194]:
df.index = df['ID']

In [197]:
df = df.drop('ID', axis = 1)

In [198]:
df1.index = df1['ID']

In [199]:
df1 = df1.drop('ID', axis = 1)

In [203]:
df1

Year,2020
ID,
19643,17
19647,16
19677,17
19681,16
19688,16
...,...
375834,19
375848,12
375884,8


In [204]:
df = df1.combine_first(df)

In [205]:
df

Year,2019,2020
ID,,
19630,7.0,NaN
19643,NaN,17.0
19647,14.0,16.0
19677,NaN,17.0
19681,18.0,16.0
...,...,...
375848,NaN,12.0
375884,7.0,8.0
375954,17.0,16.0


In [207]:
df = df[2020].combine_first(df[2019])

In [209]:
df = df[df>=15]

In [212]:
df = pd.DataFrame(df)

In [218]:
df.set_index(df['ID'])

KeyError: 'ID'

In [215]:
df.sort_values(by = [2020, index])

NameError: name 'index' is not defined

In [193]:
df1[2020].combine_first(df[2019])

Year,ID,2020
0,19643,17
1,19647,16
2,19677,17
3,19681,16
4,19688,16
...,...,...
6608,375834,19
6609,375848,12
6610,375884,8
6611,375954,16


In [116]:
df.drop_duplicates()

Year,ID,2019,2020
2,19681,18.0,16.0
3,19688,18.0,16.0
6,19710,15.0,18.0
11,19729,18.0,15.0
30,19912,16.0,16.0
...,...,...,...
6587,374718,18.0,16.0
6589,374757,15.0,19.0
6594,374801,17.0,17.0
6629,375834,16.0,19.0


In [117]:
df[2020] = df[2020].combine_first(df[2019]).sort_values()

C:\Users\roman\AppData\Local\Temp\ipykernel_1504\1277645531.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[2020] = df[2020].combine_first(df[2019]).sort_values()


In [122]:
df

Year,ID,2019,2020
2,19681,18.0,16.0
3,19688,18.0,16.0
6,19710,15.0,18.0
11,19729,18.0,15.0
30,19912,16.0,16.0
...,...,...,...
6587,374718,18.0,16.0
6589,374757,15.0,19.0
6594,374801,17.0,17.0
6629,375834,16.0,19.0


In [123]:
df = df.reindex(['ID', 2020], axis = 1)

In [126]:
df.sort_values(by = [2020, 'ID'], ascending = [True, False])

Year,ID,2020
6581,374622,15.0
6511,373002,15.0
6508,372953,15.0
6502,372899,15.0
6495,372763,15.0
...,...,...
322,23639,19.0
290,23401,19.0
146,21463,19.0
130,21408,19.0
